In [11]:
import pandas as pd
import time
import numpy as np
from pathlib import Path
import xlsxwriter
import openpyxl
import dateutil
import warnings
import recordlinkage


def runner(worksheet):
    df = pd.read_excel(raw_file_name, index_col=False)
    indexer = recordlinkage.Index()
    indexer.block('FirstName')
    candidate_links = indexer.index(df)
    
    compare_cl = recordlinkage.Compare()

    compare_cl.exact('FirstName', 'FirstName', label='FirstName')
    compare_cl.string('LastName', 'LastName', method='jarowinkler', threshold=0.75, label='LastName')
    
    features = compare_cl.compute(candidate_links, df)
    
    #print(features.head(10))
    matches = features[features.sum(axis=1) > 1]

   # print(matches)
    print(df)
    
    features.sum(axis=1).value_counts().sort_index(ascending=False)
    potential_dupes = features[features.sum(axis=1) > 1].reset_index()
    potential_dupes['Score'] = potential_dupes.loc[:, 'FirstName':'LastName'].sum(axis=1)
    
    fname = []
    lname = []
    
    for lev0 in potential_dupes['level_0']:
        #potential_dupes['link1-FirstNam] = 
        indexed = df.loc[[lev0]]
        fname.append(indexed['FirstName'].values[0])
        lname.append(indexed['LastName'].values[0])
        
        #potential_dupes.loc[[lev0]]['link0-FirstName'] = indexed['FirstName']
        #potential_dupes.loc[[lev0]]['link0-LastName'] = indexed['LastName']
        
    potential_dupes['level0-FirstName'] = fname
    potential_dupes['level0-LastName'] = lname
    
    fname1 = []
    lname1 = []
    
    for lev1 in potential_dupes['level_1']:
        #potential_dupes['link1-FirstNam] = 
        indexed = df.loc[[lev1]]
        fname1.append(indexed['FirstName'].values[0])
        lname1.append(indexed['LastName'].values[0])
        
        #potential_dupes.loc[[lev0]]['link0-FirstName'] = indexed['FirstName']
        #potential_dupes.loc[[lev0]]['link0-LastName'] = indexed['LastName']
        
    potential_dupes['level1-FirstName'] = fname1
    potential_dupes['level1-LastName'] = lname1
    
    print(potential_dupes)
    columnstor = ["level_0", "level0-FirstName", "level0-LastName", "level_1", "level1-FirstName", "level1-LastName", "FirstName", "LastName", "Score"]
    
    
    
    potential_dupes = potential_dupes.reindex(columns=columnstor)
    
    with pd.option_context('expand_frame_repr', False):
        print(potential_dupes)


        
def runner2(worksheet):
    compare_list = ['Full Name', 'Address Line1', 'Home Email']
    thresholdVal = 0.80
    
    
    df = pd.read_excel(raw_file_name, index_col=False)
    indexer = recordlinkage.Index()
    indexer.sortedneighbourhood(left_on='State Territory', window=3)
    candidate_links = indexer.index(df)
    compare_cl = recordlinkage.Compare()

    #for name in compare_list:
    #    compare_cl.string(name, name, threshold=thresholdVal, method='levenshtein', label=name)
    
    compare_cl.exact('Full Name', 'Full Name', label='Full Name')
    compare_cl.exact('Address Line1', 'Address Line1', label='Address Line1')
    compare_cl.exact('Home Email', 'Home Email', label='Home Email')
    #compare_cl.numeric('PhoneNumber', 'PhoneNumber', method='linear', label='PhoneNumber')
    

    features = compare_cl.compute(candidate_links, df)
    
    #print(features.head(10))
    #matches = features[features.sum(axis=1) > 1]
    #print(matches)
    #print(df)
    
    features.sum(axis=1).value_counts().sort_index(ascending=False)
    potential_dupes = features[features.sum(axis=1) > 0].reset_index()
    potential_dupes['Score'] = potential_dupes.loc[:, (compare_list)].sum(axis=1)
    #potential_dupes['Score2'] = potential_dupes.loc[:, 'FName':'LName'].sum(axis=1)
    
    #print(potential_dupes)
    
    index_list = []
    match_list = []
    
    for number in range(0, len(compare_list)):
        index_list.append([])
        match_list.append([])
    
    for lev0 in potential_dupes['level_0']:
        #potential_dupes['link1-FirstNam] = 
        indexed = df.loc[[lev0]]
        for number in range(0, len(compare_list)):
            index_list[number].append(indexed[compare_list[number]].values[0])
        
    
    
    for lev1 in potential_dupes['level_1']:
        #potential_dupes['link1-FirstNam] = 
        indexed = df.loc[[lev1]]
        for number in range(0, len(compare_list)):
            match_list[number].append(indexed[compare_list[number]].values[0])
    
    columnstor = ["Score", "level_0", "level_1"]
    
    for number in range(0, len(compare_list)):
        potential_dupes['|'] = ""
        potential_dupes['Index-{}'.format(compare_list[number])] = index_list[number]
        potential_dupes['Match-{}'.format(compare_list[number])] = match_list[number]
        
        columnstor.append('|')
        columnstor.append('Index-{}'.format(compare_list[number]))
        columnstor.append('Match-{}'.format(compare_list[number]))

    
    potential_dupes = potential_dupes.reindex(columns=columnstor)
    potential_dupes = potential_dupes.rename({'level_0': 'Index', 'level_1': 'Match'}, axis=1)
    

    potential_dupes = potential_dupes.sort_values(by = 'Score', ascending = False)

    
    def outputfile(dfout):
        filename = ('out_duplicates.xlsx')
        writer = pd.ExcelWriter(filename)
        dfout.to_excel(writer, engine='xlsxwriter', index=False, header=True)
        writer.save()

        #red_color = 'ffc7ce'
        #red_fill = styles.PatternFill(start_color=red_color, end_color=red_color, fill_type='solid')
        #worksheet.conditional_formatting.add('E1:B10', formatting.CellIsRule(operator='lessThan', formula=['0'], fill=red_fill))
        

    outputfile(potential_dupes)
    display(potential_dupes)
    #with pd.option_context('expand_frame_repr', False):
    #    print(potential_dupes)
        
        

#Open raw file for data access
clean_pathlist = []
pathlist = list((Path('./').glob('**/*.xlsx')))
for path in pathlist:
    if not "duplicates".casefold() in path.stem.casefold():
        clean_pathlist.append(path)

total = len(clean_pathlist)
current = 1
print("------------------------------------")
for file in clean_pathlist:
    raw_file_name = file.resolve()
    print("Total xlsx files found: {}".format(total))
    print("Starting file {}/{}: {}".format(current, total, file))
    workbook = openpyxl.load_workbook(raw_file_name)
    worksheet = workbook.active
    runner2(worksheet)
    
    path1 = 'C:\\Users\\Xukrao\\Desktop\\workbook1.xlsx'
    path2 = 'C:\\Users\\Xukrao\\Desktop\\workbook2.xlsx'

    wb2 = openpyxl.load_workbook('out_duplicates.xlsx')
    ws2 = wb2.create_sheet("original")

    for row in worksheet:
        for cell in row:
            ws2[cell.coordinate].value = cell.value

    wb2.save('out_duplicates.xlsx')
    
        
    print("Matches found, check output file for information on duplicates")
    print("Done running file {}".format(file))
    print("------------------------------------")
        
    #print("Matches found, check output file for information on duplicates")
    #print("Done running file {}".format(file))
    #print("------------------------------------")
    #current = current + 1
    

------------------------------------
Total xlsx files found: 1
Starting file 1/1: file_run.xlsx


,Score,Index,Match,|,Index-Full Name,Match-Full Name,|,Index-Address Line1,Match-Address Line1,|,Index-Home Email,Match-Home Email
1536,2,1565,1312,,Kyle Miller,Kyle Miller,,1666 32nd St NW,1666 32nd St NW,,kaugmiller@gmail.com,NaN
1417,2,1355,882,,Amjad Ali Alqahtani,Amjad Ali Alqahtani,,4547 Indian Rock Ter NW,4547 Indian Rock Ter NW,,NaN,NaN
694,2,1538,1339,,Sarena Noelani Young,Sarena Noelani Young,,1707 N Troy St,1707 N Troy St,,NaN,NaN
1474,2,1424,1393,,Helena Cristina Dill,Helena Cristina Dill,,2425 L ST NW,2425 L ST NW,,NaN,hcd17@georgetown.edu
1045,2,1781,1765,,Ali Marhoon,Ali Marhoon,,1800 N Lynn St,1800 N Lynn St,,aam261@georgetown.edu,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
535,1,1376,783,,John Robert Pruett,Kyu Lee,,1531 N Pierce St,1531 N Pierce St,,jrp157@georgetown.edu,NaN
534,1,1376,770,,John Robert Pruett,Margaret Jacqueline Hendricks,,1531 N Pierce St,1531 N Pierce St,,jrp157@georgetown.edu,NaN
533,1,1376,405,,John Robert Pruett,Danielle Morency,,1531 N Pierce St,1531 N Pierce St,,jrp157@georgetown.edu,NaN
532,1,1376,337,,John Robert Pruett,Jie Cheng,,1531 N Pierce St,1531 N Pierce St,,jrp157@georgetown.edu,NaN


Matches found, check output file for information on duplicates
Done running file file_run.xlsx
------------------------------------


In [129]:
pd.options.display.float_format = '{:.0f}'.format

In [ ]:
pip install Levenshtein-search

In [1]:
pip install pandas_dedupe

  Using cached dedupe-2.0.13.tar.gz (70 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached fastcluster-1.2.4-cp39-cp39-win_amd64.whl (36 kB)
  Using cached categorical_distance-1.9-py3-none-any.whl (3.3 kB)
  Using cached Levenshtein_search-1.4.5.tar.gz (7.8 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached haversine-2.5.1-py2.py3-none-any.whl (6.1 kB)
  Using cached highered-0.2.1-py2.py3-none-any.whl (3.3 kB)
  Using cached dedupe_hcluster-0.3.9-cp39-cp39-win_amd64.whl (170 kB)
  Using cached DoubleMetaphone-1.1-cp39-cp39-win_amd64.whl (28 kB)
  Using cached affinegap-1.12-cp39-cp39-win_amd64.whl (16 kB)
  Using cached de

  error: subprocess-exited-with-error
  
  exit code: 1
  
  [46 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.9
  creating build\lib.win-amd64-3.9\dedupe
  copying dedupe\api.py -> build\lib.win-amd64-3.9\dedupe
  copying dedupe\backport.py -> build\lib.win-amd64-3.9\dedupe
  copying dedupe\blocking.py -> build\lib.win-amd64-3.9\dedupe
  copying dedupe\canonical.py -> build\lib.win-amd64-3.9\dedupe
  copying dedupe\canopy_index.py -> build\lib.win-amd64-3.9\dedupe
  copying dedupe\clustering.py -> build\lib.win-amd64-3.9\dedupe
  copying dedupe\convenience.py -> build\lib.win-amd64-3.9\dedupe
  copying dedupe\core.py -> build\lib.win-amd64-3.9\dedupe
  copying dedupe\datamodel.py -> build\lib.win-amd64-3.9\dedupe
  copying dedupe\index.py -> build\lib.win-amd64-3.9\dedupe
  copying dedupe\labeler.py -> build\lib.win-amd64-3.9\dedupe
  copying dedupe\levenshtein.py -> build\lib.win-amd64-3.9\dedupe
  copying 

In [ ]:
pip install --upgrade pip setuptools wheel

In [24]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [63]:
for number in range(0, 3):
    print(number)

0
1
2
